This is actually a section of short exercises, so we'll keep it here instead of inside the projects folder for simplicity.

Consider the following classes:

In [1]:
class Stock:
    def __init__(self, symbol, date, open, high, low, close, volume):
        self.symbol = symbol
        self.date = date
        self.open = open
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume
        
class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume

#### Exercise 1

Given the above class, write a custom `JSONEncoder` class to **serialize** dictionaries that contain instances of these particular classes. Keep in mind that you will want to deserialize the data too - so you will need some technique to indicate the object type in your serialization.

For example you may have an object such as this one that needs to be serialized:

In [2]:
from datetime import date, datetime
from decimal import Decimal

activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22), 
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22), 
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

#### Solution

Here is a solution different to the one provided in the video. It is based off the solution found in the comments under the Solution video (61) which uses `jsonschema` for validation and `@singledispatch` for handling serialisation of the different types. 

To keep things simple, we'll only implement the `@singledispatch` side of that solution.

Here's our plan:

1. Implement `as_dict()` methods to both of our classes - this will be used to tell our JSONEncoder what to do when it comes across the unfamiliar `Stock`/`Trade` "type". We will not typecast any values yet; `self.date` will be a `date` type object.

2. Add an additional key to the `as_dict()` methods to indicate what class this data came from - this will help with serialising it back to the correct class in the next exercise.

3. Create a `CustomJSONEncoder` class and inside that, create a `@singledispatchmethod` function  for which we'll register all of our types with their desired serialisation behaviour.

4. In our `@singledispatchmethod` function, return str(arg) - this is the default behaviour if an unexpected type is received.

6. Return this `@singledispatchmethod` function in our `default(self, arg)` method inside our `CustomJSONEncoder` class - when `json.dumps()` is called and a type that json doesn't know how to serialise appears, this `default(self, arg)` function will be called to the rescue.

In [3]:
# STEP 1, 2

class Stock:
    def __init__(self, symbol, date, open, high, low, close, volume):
        self.symbol = symbol
        self.date = date
        self.open = open
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume

    def as_dict(self):
        return dict(symbol=self.symbol, date=self.date, open=self.open, high=self.high, low=self.low, close=self.close, volume=self.volume, object=self.__class__.__name__)
        
class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume

    def as_dict(self):
        return dict(symbol=self.symbol, timestamp=self.timestamp, order=self.order, price=self.price, commission=self.commission, volume=self.volume, object=self.__class__.__name__)

In [4]:
import json
from datetime import date, datetime
from decimal import Decimal

activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22), 
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22), 
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

In [5]:
# STEP 3, 4, 5, 6

from functools import singledispatchmethod
import json
from datetime import date, datetime
from decimal import Decimal

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, arg):
        return self.serialise_obj(arg)

    @singledispatchmethod
    def serialise_obj(self, obj):
        return str(obj)  # default behaviour

    @serialise_obj.register(date)
    def _(self, arg):
        return arg.isoformat()

    @serialise_obj.register(datetime)
    def _(self, arg):
        return arg.isoformat()

    @serialise_obj.register(Decimal)
    def _(self, arg):
        return str(arg)

    @serialise_obj.register(Stock)
    @serialise_obj.register(Trade)
    def _(self, arg):
        return arg.as_dict()



In [6]:
print(json.dumps(activity, cls=CustomJSONEncoder, indent=2))

{
  "quotes": [
    {
      "symbol": "TSLA",
      "date": "2018-11-22",
      "open": "338.19",
      "high": "338.64",
      "low": "337.60",
      "close": "338.19",
      "volume": 365607,
      "object": "Stock"
    },
    {
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": "176.66",
      "high": "177.25",
      "low": "176.64",
      "close": "176.78",
      "volume": 3699184,
      "object": "Stock"
    },
    {
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": 4493689,
      "object": "Stock"
    }
  ],
  "trades": [
    {
      "symbol": "TSLA",
      "timestamp": "2018-11-22T10:05:12",
      "order": "buy",
      "price": "338.25",
      "commission": "9.99",
      "volume": 100,
      "object": "Trade"
    },
    {
      "symbol": "AAPL",
      "timestamp": "2018-11-22T10:30:05",
      "order": "sell",
      "price": "177.01",
      "commission"

#### Exercise 2

Write code to reverse the serialization you just created. Write a custom decoder that can deserialize a JSON structure containing `Stock` and `Trade` objects. 

#### Solution

Here's our plan:

1. Update the `Stock` and `Trade` classes (the ones made after Exercise 1) with an `__eq__` method matching on all elements (note that we can use compare equal on the `as_dict` methods of each class instance instead of individually comparing each property. This step is so we can ensure that we retrieve the original object after serialising + deserialising.
2. Write a utility function `decode_stock()` that takes a dictionary assuming it used to be a `Stock` object and recreate a `Stock` object, parsing each key's value back to its original type.
3. Do exactly the same for `Trade`.
4. Write a utility function called `decode_financials` which will take a dictionary and look for the `'object'` key. If `d['object'] == 'Stock'`, call `decode_stock()`, and likewise for `'Trade'`. If it's neither, return the original dictionary. 
5. Subclass `json.JSONDecoder` and override the `decode(arg)` method.
6. Write another utility function called `parse_financials()` - this will be our recursive function that will be called in `decode()`. Here's how it will deal with its argument:
    - if it's a dictionary, call `decode_financials()` - this will return a `Stock` or `Trade` object, or the original dictionary.
    - if it's still a dictionary (from calling the above), call this function on all values in the dictionary - there could be a `Stock`/`Trade` object within.
    - if it's a list, enumerate it and call the this function on each element, replacing the original element.
    - if it's anything else, return it as is.
7. In the `decode()` method, first run `json.loads(arg)` to handle the rough initial parsing and then run your `parse_financials()`


In [7]:
# STEP 1
class Stock:
    def __init__(self, symbol, date, open, high, low, close, volume):
        self.symbol = symbol
        self.date = date
        self.open = open
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume

    def as_dict(self):
        return dict(symbol=self.symbol, date=self.date, open=self.open, high=self.high, low=self.low, close=self.close, volume=self.volume, object=self.__class__.__name__)

    def __eq__(self, other):
        return isinstance(other, Stock) and self.as_dict() == other.as_dict()
        
class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume

    def as_dict(self):
        return dict(symbol=self.symbol, timestamp=self.timestamp, order=self.order, price=self.price, commission=self.commission, volume=self.volume, object=self.__class__.__name__)

    def __eq__(self, other):
        return isinstance(other, Trade) and self.as_dict() == other.as_dict()

In [8]:
# STEP 2
from datetime import datetime
from decimal import Decimal

def decode_stock(d):
    return Stock(
        symbol=str(d['symbol']),  # not necessary since json.loads would've already ensured this, but nice for readability/reminder
        date=datetime.strptime(d['date'], '%Y-%m-%d').date(),
        open=Decimal(d['open']),
        high=Decimal(d['high']),
        low=Decimal(d['low']),
        close=Decimal(d['close']),
        volume=int(d['volume'])  # not necessary since ...
    )

In [9]:
# STEP 3
def decode_trade(d):
    return Trade(
        symbol=str(d['symbol']),  # not necessary since json.loads would've already ensured this, but nice for readability/reminder
        timestamp=datetime.strptime(d['timestamp'], '%Y-%m-%dT%H:%M:%S'),
        order=str(d['order']),  # not necessary since ...
        price=Decimal(d['price']),
        commission=Decimal(d['commission']),
        volume=int(d['volume'])  # not necessary since ...
    )

In [10]:
# STEP 4
def decode_financials(d):
    object_type = d.get('object', None)
    if object_type == 'Stock':
        return decode_stock(d)
    elif object_type == 'Trade':
        return decode_trade(d)
    else:
        return d

In [11]:
# STEP 5, 6, 7
import json

class CustomJSONDecoder(json.JSONDecoder):
    def decode(self, arg):
        obj = json.loads(arg)
        return self.parse_financials(obj)

    def parse_financials(self, obj):
        if isinstance(obj, dict):
            obj = decode_financials(obj)
            if isinstance(obj, dict):
                for key, value in obj.items():
                    obj[key] = self.parse_financials(value)

            
        elif isinstance(obj, list):
            for idx, elem in enumerate(obj):
                obj[idx] = self.parse_financials(elem)

        return obj

Now that we're done writing the setup, we will take the initial data, serialise it, deserialise it, and then compare it to initial. 

In [13]:
activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22), 
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22), 
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

In [14]:
class CustomJSONEncoder(json.JSONEncoder):
    def default(self, arg):
        return self.serialise_obj(arg)

    @singledispatchmethod
    def serialise_obj(self, obj):
        return str(obj)  # default behaviour

    @serialise_obj.register(date)
    def _(self, arg):
        return arg.isoformat()

    @serialise_obj.register(datetime)
    def _(self, arg):
        return arg.isoformat()

    @serialise_obj.register(Decimal)
    def _(self, arg):
        return str(arg)

    @serialise_obj.register(Stock)
    @serialise_obj.register(Trade)
    def _(self, arg):
        return arg.as_dict()

In [19]:
encoded = json.dumps(activity, cls=CustomJSONEncoder, indent=2)
decoded = json.loads(encoded, cls=CustomJSONDecoder)
print(decoded == activity)

True
